Fase di import delle librerie

In [ ]:
import pandas as pd
import numpy as np
import re

Importazione dei due db <br>
**Netflix:** database riferito a tutte le serie Netflix aggiornato al 2019 <br>
**IMDB:** database riferito ai rating del sito IMDB aggiornato al 2020 <br>

In [ ]:
netflix=pd.read_csv('netflix_titles.csv')
IMDB=pd.read_csv('series_data.csv')

In [ ]:
netflix.head()

In [ ]:
IMDB.head()

**INTEGRAZIONE DEI DUE DATABASE** <br>
Merge dei due database, in formato csv, prendendo in considerazione solo le serie tv.

Prima dell'integrazione, abbiamo effettuato una normalizzazione sulle colonne dei titoli delle serie tv.



In [ ]:
IMDB['Series_Title_norm'] = IMDB['Series_Title'].str.lower()

In [ ]:
netflix['title_norm'] = netflix['title'].str.lower()

In [ ]:
unione=pd.merge(netflix, IMDB, left_on = 'title_norm', right_on = 'Series_Title_norm')

In [ ]:
unione.head()

In [ ]:
unione=unione[unione['type']=='TV Show']
#sono solamente 2 i film, solo serie tv in comune sono 392

In [ ]:
unione.drop(['Series_Title', 'Poster_Link','listed_in','Star1', 'Star2', 'Star3', 'Star4','release_year','Runtime_of_Series','date_added','description', 'Overview' ], axis=1, inplace=True)


In [ ]:
unione.head()

**INTEGRAZIONE CON JSON**

Fase di importazione delle librerie

In [ ]:
import json
import re
import tweepy

In [ ]:
pip install tweepy

In [ ]:
#Lettura file json contenente i tweet
tweet=pd.read_json("finale.json")

In [ ]:
#rimozione carattere 'oid' aggiunto da mongo 
tweet["_id"] = tweet["_id"].map(lambda x: x['$oid'])

In [ ]:
tweet.head()

FUNZIONE PER LA PULIZIA 

In [ ]:
#Importazione di emoji: un package in cui sono contenute le emoji
import string
import re
import emoji


In [ ]:
def cleaner(tweet):
    tweet = re.sub("@[A-Za-z0-9]+","",tweet) #rimuove @
    tweet = re.sub("~[A-Za-z0-9]+","",tweet) #rimuove @
    tweet = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", tweet) #Rimuove il link
    tweet = " ".join(tweet.split())
    tweet = ''.join(c for c in tweet if c not in emoji.UNICODE_EMOJI) #Rimuove le emojis
    tweet = tweet.replace("#", "").replace("_", " ") #Rimuove il simbolodell'hashtag ma tiene il testo dell'hashtag
    #per i simboli che noon vengono presi dalle istruzioni precedenti 
    emoj = re.compile("[" 
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"  #emojii della forbice 
        u"\U00002702-\U000027B0"  #
        u"\U000024C2-\U0001F251"  #accept
        u"\U0001f926-\U0001f937"  #facepalm/SHRUG
        u"\U00010000-\U0010ffff"#stop
        "U+007E" #tilde 
        u"\u2640-\u2642" #simbolo male
        u"\u2600-\u2B55"#ancora-cerchio nero 
        u"\u200d" #ZERO WIDTH JOINER'
        u"\u23cf" #simbolo eject
        u"\u23e9" #simbolo avanti 
        u"\u231a" #orologio
        "U+20AC" #a cappuccio
        "\ufe0f"  # dingbats
        u"\u3030" #baffi
                      "]+", re.UNICODE)
    return re.sub(emoj, '', tweet)
    return tweet

#link delle emoji 
#http://unicode.org/reports/tr51/tr51-12.html#Identification

In [ ]:
#applico la funzione cleaner alla colonna text e salvo tutto in una colonna text_cleaned
tweet['text_cleaned'] = tweet['text'].map(lambda x: cleaner(x))

In [ ]:
tweet['text_cleaned']

In [ ]:
#applico la funzione cleaner_punctuation alla colonna text_cleaned e salvo tutto in una colonna text_cleaned_punctuation
def cleaner_punctuation(tweet):
    exclude = set(string.punctuation)
    tweet = ''.join(ch for ch in tweet if ch not in exclude)
    return tweet

In [ ]:
tweet['text_cleaned_punctuation'] = tweet['text_cleaned'].map(lambda x: cleaner_punctuation(x))

In [ ]:
tweet['text_cleaned_punctuation']

In [ ]:
#ricerca dei titoli delle serie tv presenti nel testo del tweet con creazione della colonna title per facilitare l'integrazione finale
i=0
tweet['title']=''
for testo in tweet['text_cleaned_punctuation']:
    titolo_match=''
    count_match=0
    for titolo in unione['title']:
        if titolo in testo:
            titolo_match=titolo
            count_match+=1
    
    if (count_match>1 or count_match==0):
        tweet.drop(i, inplace=True)
    else: 
        tweet['title'][i]=titolo_match
    i+=1

In [ ]:
#integrazione finale tra dataset twitter e unione precedente di IMDB e Netflix
finale=pd.merge(unione, tweet, left_on='title', right_on='title')

In [ ]:
finale.drop(['Unnamed: 0'], axis=1, inplace=True)

In [ ]:
data=finale.to_dict(orient='records')

In [ ]:
from itertools import groupby
from operator import itemgetter
import json

In [ ]:
jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10

In [ ]:
# aggregazione di tutti i tweet riferiti ad una specifica serie tv
move = '_id','text','hashtags','score1','score2','text_cleaned__punctuation'

key = itemgetter('show_id')
data.sort(key=key)

result = []
for show_id,group in groupby(data,key=key):
    for i,item in enumerate(group):
        if i == 0: 
            current = item 
            result.append(current)
            current['tweet'] = []

        current_tweet = {} 
        current['tweet'].append(current_tweet)
        for movekey in move:
            current_tweet[movekey] = item[movekey]
            if i == 0: 
                del current[movekey]

print(json.dumps(result,indent=2))

In [ ]:
import json
with open('integrazione.json', 'w') as f:
    f.write(result + "\n")